In [43]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from kneed import KneeLocator

import string
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinamazur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ekaterinamazur/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [120]:
#read the pkl file for movies
df = pd.read_pickle('./data/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./data/full_shows.pkl')

In [123]:
#read json file activity for movies
# activity_movie = pd.read_json('./movie_activities.json')
# activity_shows = pd.read_json('./show_activities.json')
activities = pd.read_json('./data/activities.json')

In [124]:
activities.head()
activities.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
0,178,178,178
1,176,176,176
2,196,196,196
3,194,194,194
4,159,159,159
5,169,169,169
6,195,195,195
7,190,190,190
8,214,214,214


## TF-IDF + kmeans clustering

In [95]:
stop_words = stopwords.words('english')
def clean_text(x):
    x = x.lower()
    x = x.translate(str.maketrans('', '', string.punctuation))
    # tokenize
    x = nltk.word_tokenize(x)
    x = [token for token in x if token not in stop_words]
    # lemmatization and pass string back
    x = ' '.join([lemmatizer.lemmatize(w) for w in x])
    return x

In [125]:
df['features'] = df['description'].apply(lambda x: clean_text(x))
df_2['features'] = df_2['description'].apply(lambda x: clean_text(x))

In [126]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 80)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


None


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/kneed/knee_locator.py:311: UserWarning: No knee/elbow found
  warnings.warn("No knee/elbow found")


In [127]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['features'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

8


In [173]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters
df.head()

,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,cluster
0,The Way We Were,0,Movies,The Way We Were,NaN,The Way We Were,A love story that begins with the attraction o...,A love story that begins with the attraction o...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zy/ZY933...,[['Director Sydney Pollack']],"[abc1, abc2, drama, classic, film, feature-len...",2021-06-25 07:00:00,PG,6802.0,NaN,,love story begin attraction opposite katie hub...,8
1,Starman,1,Movies,Starman,NaN,Starman,An alien stranded on Earth clones himself into...,An alien stranded on Earth clones himself into...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,[['Director John Carpenter']],"[abc1, abc2, drama, sci-fi, fantasy, romance, ...",2021-06-25 07:00:00,PG,6612.0,NaN,,alien stranded earth clone form young woman de...,3
2,Mr Deeds Goes To Town,2,Movies,Mr Deeds Goes To Town,NaN,Mr Deeds Goes To Town,A small-town poet inherits a vast fortune and ...,A small-town poet inherits a vast fortune and ...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,"[['Cast Gary Cooper', 'Jean Arthur', 'Douglas ...","[abc1, abc2, drama, comedy-drama, film, featur...",2021-06-25 07:00:00,G,6649.0,NaN,,smalltown poet inherits vast fortune find new ...,6
3,On The Road,3,Movies,On The Road,NaN,On The Road,Aspiring writer Sal Paradise has his world roc...,Aspiring writer Sal Paradise has his world roc...,"[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW291...,[['Director Walter Salles']],"[abc1, abc2, usa, drama, adaptation, adventure...",2021-06-01 07:00:00,MA,8014.0,NaN,,aspiring writer sal paradise world rocked foll...,2
4,The Boys Are Back,4,Movies,The Boys Are Back,NaN,The Boys Are Back,"Journalist Joe Warr and his two sons, from dif...","Journalist Joe Warr and his two sons, from dif...","[ABC TV, ABC TV Plus, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,[['Director Scott Hicks']],"[abc1, abc2, drama, comedy-drama, film, single...",2021-06-11 07:00:00,M,5950.0,NaN,,journalist joe warr two son different marriage...,4


In [129]:
# Cluster the documents using KMeans
num_clusters_shows = 7
km_shows = KMeans(n_clusters=num_clusters_shows)
km_shows.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km_shows.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.reset_index(inplace=True)
df_2.head()

,index,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[primary-humanities, education, abc3, refugee-...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,5
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[lismore, news, childcare, flooding, abc-faves...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,2
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, parachuting, humphries, candidates, sus...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,2
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,0
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,4


In [174]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'
diverse = 'queer|lbgtqi|diversity|inclusion|disability|aboriginal|race|racism|indigenous|equal-opportunit'

In [175]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
diversity_library = df[df['tags2'].str.contains(diverse) | df['description'].str.contains(diverse) | df['description2'].str.contains(diverse) | df['title'].str.contains(diverse)]
diversity_library_shows= df_2[df_2['tags2'].str.contains(diverse) | df_2['description'].str.contains(diverse) | df_2['description2'].str.contains(diverse) | df_2['title'].str.contains(diverse)
]

In [176]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index'], inplace=True)
df_2.head()

,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features,cluster
0,0,10000,199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[[]],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[primary-humanities, education, abc3, refugee-...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW073...,journey school special type life experience sh...,5
1,1,10001,7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[[]],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[lismore, news, childcare, flooding, abc-faves...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC220...,leigh sale present australia leading nightly p...,2
2,2,10002,7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[[]],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...",[ABC NEWS],"[mark, parachuting, humphries, candidates, sus...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN211...,satirist mark humphries brings unique perspect...,2
3,3,10003,7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[[]],2022-03-08 20:00:00,None,[1030.0],[ABC NEWS],"[news24, news]",https://cdn.iview.abc.net.au/thumbs/i/ns/NS224...,revisit 730s compelling conversation australia...,0
4,4,10004,72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[[]],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW243...,animal kingdom continually astounds u showcase...,4


In [177]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: artist connection explores story heart personal man aboriginal political neighbour share cancer challenge
Cluster 1: school play special join child episode emma country celebrating family safe australia stick
Cluster 2: world hand desert isolated wash culture song power ancient greatest island new film
Cluster 3: great journey life way sea earth chronicle universe 2014 raising set year seagull
Cluster 4: begin 2012 best son friend set novel lover affair life father england young
Cluster 5: david attenborough attenboroughs making monster sir human home flying wildest truth set series
Cluster 6: new australian father grief york city portrait intimate club daughter navigating home bush
Cluster 7: people disability film story christmas beach tell stage day live international summer short
Cluster 8: love young stage julia life family couple book successful actress turn 2004 donaldson
Cluster 9: work battle 2013 woman story australia biggest sexual company year place group inside


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [178]:
# Print the top 10 terms for each cluster
order_centroids_shows = km_shows.cluster_centers_.argsort()[:, ::-1]
terms_shows = tfidf_vectorizer_shows.get_feature_names()
for i in range(num_clusters_shows):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_shows[i, :13]:
        print(' %s' % terms_shows[ind], end='')
    print()

Cluster 0: australian year life story art team science people action crime teacher join skill
Cluster 1: new family friend adventure world life great australian play exploring history series member
Cluster 2: australia political best week look national australian team story panel people country election
Cluster 3: series adventure book fun kid based little love world word explore musical story
Cluster 4: big animal friend australia world ted summer question adventure join attenborough emergency unique
Cluster 5: stage world life english child different series early science history challenge geography math
Cluster 6: news abc issue story reporting brings today original exclusive presented matter live investigation


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [179]:
joined_activity = pd.merge(df, activities, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
2,311.0,Comedy,Intimate Encounters: 20 Years On,"In 1988, an exhibition caused shockwaves with ...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK209...,[['Director Dieter Knierim']],"'abc1', 'docs', 'short', 'arts', 'abcarts', 's...",1988 exhibition caused shockwaves photo people...,7.0,311,Dislike,30
3,340.0,Education,Carbon And Water Cycles In The Rainforest,"Filmed in the Amazon and Borneo, this film ill...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW168...,[[]],"'abc3', 'education', 'secondary-geography'",filmed amazon borneo film illustrates explains...,0.0,340,Like,21
4,127.0,Movies,Peggy Sue Got Married,A woman on the verge of her marriage falling a...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,[['Director Francis Ford Coppola']],"'abc1', 'abc2', 'drama', 'comedy-drama', 'clas...",woman verge marriage falling apart attends hig...,1.0,127,Like,16
5,199.0,Kids,See Us Coming Together: A Sesame Street Special,"It's ""Neighbour Day"" on Sesame Street and ever...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW325...,[[]],'abc4kids',neighbour day sesame street everyone something...,0.0,199,Like,4
6,157.0,Family,Freeman,The story of a nation coming together around I...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[['Director Laurence Billiet']],"'abc1', 'australia', 'indigenous', 'docs', 'fa...",story nation coming together around indigenous...,9.0,157,Like,26


In [180]:
joined_activity_shows = pd.merge(df_2, activities, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity_shows['tags2'] = joined_activity_shows['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity_shows.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity_shows.drop([ 'description2', 'publication_date', 'rating', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity_shows.head()

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
3002,443.0,10443.0,Science Max! Experiments At Large,Education,Have you ever done a science experiment and wo...,[[]],"'secondary-science', 'education', 'abc3', 'pri...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW115...,ever done science experiment wondered would li...,3.0,10443,Like,23
3003,565.0,10565.0,Titch And Ted Do Maths,Education,This live action comedy series introduces odd ...,[[]],"'3scienceweek', '3science-maths', 'education',...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW036...,live action comedy series introduces odd coupl...,4.0,10565,Dislike,13
3004,205.0,10205.0,Going Country,Documentary,Justine Clarke takes a road trip into the hear...,[['Host Justine Clarke']],"'diversity', 'regional', '90', 'factual', 'doc...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,justine clarke take road trip heart australian...,0.0,10205,Dislike,12
3005,42.0,10042.0,Annika,Drama,Detective Chief Inspector Annika Strandhed has...,"['Ukweli Roach'], ['Cast Nicola Walker', 'Kati...","'mystery', 'strong-female-lead', 'thriller', '...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW261...,detective chief inspector annika strandhed bec...,1.0,10042,Like,24
3006,367.0,10367.0,Odd Squad Healthy Habits,Family,Odd Squad's The Big O shares a few top tips fo...,[[]],"'family-viewing', 'abc3'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW228...,odd squad big share top tip physical distancin...,4.0,10367,Like,17


In [181]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes.sort_values(by=['user_id'])

,id,category,title,description,image,directors_actors,tags2,features,cluster,content_id,activity,user_id
2228,321.0,Comedy,Deadly Family Portraits: Electric Mimili,Robert Fielding and Zaachariaha Fielding tell ...,https://cdn.iview.abc.net.au/thumbs/i/ac/AC183...,[[]],"'abc1', 'abcarts', 'indigenious', 'docs', 'art...",robert fielding zaachariaha fielding tell stor...,7.0,321,Like,0
2048,147.0,Family,The Snail And The Whale,"A beautiful tale of an unlikely friendship, a ...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW177...,"[['Director Max Lang', 'Daniel Snaddon']]","'abc4kids', '4featured', '4feature', '4family'...",beautiful tale unlikely friendship tour world ...,8.0,147,Like,0
2007,127.0,Movies,Peggy Sue Got Married,A woman on the verge of her marriage falling a...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW302...,[['Director Francis Ford Coppola']],"'abc1', 'abc2', 'drama', 'comedy-drama', 'clas...",woman verge marriage falling apart attends hig...,1.0,127,Like,0
1553,150.0,Family,The Making Of Flying Monsters With David Atten...,A behind-the-scenes look at the making of the ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW259...,[['Host Sir David Attenborough']],"'abc2', 'uk', 'docs', 'factual', 'nature', 'wi...",behindthescenes look making documentary david ...,5.0,150,Like,0
2939,29.0,Movies,An Accidental Soldier,"On the Western Front in WWI, an Australian on ...",https://cdn.iview.abc.net.au/thumbs/i/dr/DR120...,[['Director Rachel Ward']],"'abc1', 'drama', 'film', 'war', 'love', 'roman...",western front wwi australian run find unexpect...,8.0,29,Like,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2439,245.0,Documentary,Poleng,Told through contemporary movement and Balines...,https://cdn.iview.abc.net.au/thumbs/i/do/DO202...,[[]],"'abc1', 'docs', 'arts', 'australia', 'dance', ...",told contemporary movement balinese dance pole...,0.0,245,Like,32
1638,65.0,Movies,Murray Whelan: The Brush Off,Political adviser to Australia's Minister of t...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director Sam Neill']],"'abc1', 'australia', 'drama', 'crime', 'crime-...",political adviser australia minister art murra...,8.0,65,Like,32
2443,327.0,Comedy,Doug The Human,An alien kidnaps a human man and turns him int...,https://cdn.iview.abc.net.au/thumbs/i/ip/IP180...,[['Director Gary Hamaguchi']],"'abc1', 'arts', 'australia', 'drama', 'regiona...",alien kidnaps human man turn pet,8.0,327,Like,32
1128,178.0,Family,The Making Of The Penguin King With David Atte...,A behind-the-scenes look at the documentary wh...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW259...,[['Host Sir David Attenborough']],"'abc2', 'uk', 'docs', 'factual', 'animals', 'n...",behindthescenes look documentary follows lone ...,8.0,178,Like,32


In [182]:
#select rows where activity is 'Like'
expat_data_shows_likes = joined_activity_shows[joined_activity_shows['activity'] == 'Like']
expat_data_shows_likes.sort_values(by=['user_id'])

,level_0,id,title,category,description,directors_actors,tags2,image,features,cluster,content_id,activity,user_id
4482,129.0,10129.0,Cubs,Kids,There are 8.7 millions of animal species in th...,[[]],"'abc4kids', '4puggle'",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW226...,87 million animal specie world cub animation t...,4.0,10129,Like,0
3535,249.0,10249.0,Insiders: The Interviews,Panel Discussion,Interviews with leading politicians about the ...,[[]],"'interviews', 'latest', 'current-affairs', 'ne...",https://cdn.iview.abc.net.au/thumbs/i/nn/NN220...,interview leading politician week politics,2.0,10249,Like,0
3789,621.0,10621.0,triple j Like A Version,Comedy,A popular segment recorded at triple j studios...,[[]],"'live-music', 'like-a-version', 'pop-music', '...",https://cdn.iview.abc.net.au/thumbs/i/iv/IV220...,popular segment recorded triple j studio give ...,0.0,10621,Like,0
5815,201.0,10201.0,Giggle And Hoot,Kids,Jimmy Giggle and all his owl pals have awesome...,[[]],"'4lower', 'abc4kids', '4best', 'childrens'",https://cdn.iview.abc.net.au/thumbs/i/ck/CK183...,jimmy giggle owl pal awesome adventure fun tog...,0.0,10201,Like,0
4264,192.0,10192.0,Further Back In Time For Dinner (Simplified Ch...,Family,An ordinary family travels back in time 120 ye...,[[]],"'family-viewing', 'abc1', 'docs'",https://cdn.iview.abc.net.au/thumbs/i/iv/IV201...,ordinary family travel back time 120 year imme...,1.0,10192,Like,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5511,558.0,10558.0,This Place: Artist Series,Comedy,Six Aboriginal and Torres Strait Islander arti...,[[]],"'diversity', 'regional', 'a:older-sink-dink', ...",https://cdn.iview.abc.net.au/thumbs/i/ra/RA191...,six aboriginal torres strait islander artist n...,2.0,10558,Like,32
5866,109.0,10109.0,Catalyst,Documentary,Exploring the forefront of science and technol...,[[]],"'natural-world', 'melbourne', 'agriculture', '...",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,exploring forefront science technology catalys...,0.0,10109,Like,32
4231,192.0,10192.0,Further Back In Time For Dinner (Simplified Ch...,Family,An ordinary family travels back in time 120 ye...,[[]],"'family-viewing', 'abc1', 'docs'",https://cdn.iview.abc.net.au/thumbs/i/iv/IV201...,ordinary family travel back time 120 year imme...,1.0,10192,Like,32
4815,286.0,10286.0,Lest We Forget What? Education Shorts,Education,A young woman investigates Australia's remembr...,[[]],"'all-episodes', 'education', 'abc3', 'history'...",https://cdn.iview.abc.net.au/thumbs/i/zx/ZX996...,young woman investigates australia remembrance...,5.0,10286,Like,32


In [183]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,0,10.0
1,0,7.0
2,0,8.0
3,1,5.0
4,1,10.0
...,...,...
94,31,8.0
95,31,6.0
96,32,0.0
97,32,5.0


In [184]:
expat_data_shows_likes.sort_values(by=['user_id'])
#top-3 clusters for each user in expat_data_likes
top3_show = expat_data_shows_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1,
                                                                                                              drop=True)
#make from series to dataframe
top3_show = top3_show.to_frame()
#drop the 3rd column
top3_show = top3_show.drop(columns=['cluster'])
#make user_id column as index
top3_show = top3_show.reset_index()
top3_show

,user_id,cluster
0,0,0.0
1,0,2.0
2,0,5.0
3,1,0.0
4,1,5.0
...,...,...
94,31,2.0
95,31,3.0
96,32,0.0
97,32,2.0


In [185]:
#join top 3 clusters for each user with expat_library
recommended= pd.merge(top3, expat_library, on=['cluster'], how='left')
recommended

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,10.0,Designing A Legacy,223,Documentary,Designing A Legacy,NaN,Designing A Legacy,"Comedian, architecture enthusiast and design n...","Comedian, architecture enthusiast and design n...","[ABC TV, LIFESTYLE, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/fa/FA190...,[['Host Tim Ross']],"[abc1, australia, lifestyle, home, creative, a...",2021-02-02 21:03:51,M,3422.0,NaN,,comedian architecture enthusiast design nerd t...
1,0,10.0,Fighting Spirit: Wheeling Diggers' Invictus Ga...,251,Documentary,Fighting Spirit: Wheeling Diggers' Invictus Ga...,NaN,Fighting Spirit: Wheeling Diggers' Invictus Ga...,This deeply moving documentary goes behind-the...,This deeply moving documentary goes behind-the...,"[ABC TV, SPORT, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,[[]],"[abc1, aussie, sport, docs, factual, inspirati...",2018-10-16 21:28:00,M,3442.0,NaN,,deeply moving documentary go behindthescenes i...
2,0,10.0,Sisters Of War,285,Drama,Sisters Of War,NaN,Sisters Of War,Inspired by the remarkable true story of two e...,Inspired by the remarkable true story of two e...,"[ABC TV, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR090...,[['Director Brendan Maher']],"[abc1, drama, history, a:established-parents, ...",2021-06-11 07:00:00,M,5725.0,NaN,,inspired remarkable true story two extraordina...
3,0,7.0,Can We Save The Reef?,152,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
4,0,7.0,Christmas Sounds Better This Year,238,Documentary,Christmas Sounds Better This Year,NaN,Christmas Sounds Better This Year,Every Christmas the same old songs are played ...,Every Christmas the same old songs are played ...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/rn/RN181...,[['Host Kumi Taguchi']],"[abc1, australia, docs, factual, music, people...",2019-12-17 22:35:13,G,3538.0,NaN,,every christmas old song played got talented p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,32,8.0,An Accidental Soldier,29,Movies,An Accidental Soldier,NaN,An Accidental Soldier,"On the Western Front in WWI, an Australian on ...","On the Western Front in WWI, an Australian on ...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR120...,[['Director Rachel Ward']],"[abc1, drama, film, war, love, romance, adapta...",2021-06-11 07:00:00,M,5502.0,NaN,,western front wwi australian run find unexpect...
509,32,8.0,Murray Whelan: The Brush Off,65,Movies,Murray Whelan: The Brush Off,NaN,Murray Whelan: The Brush Off,Political adviser to Australia's Minister of t...,Political adviser to Australia's Minister of t...,"[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director Sam Neill']],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5436.0,NaN,,political adviser australia minister art murra...
510,32,8.0,Murray Whelan: Stiff,139,Movies,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[['Director John Clarke']],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,NaN,,murray whelan young political adviser must pla...
511,32,8.0,Stargazing: Moon and Beyond,167,Family,Stargazing: Moon and Beyond,NaN,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,Stargazing celebrates the 50th ann

In [186]:
#shows
recommended_shows= pd.merge(top3_show, expat_library_shows, on=['cluster'], how='left')
recommended_shows

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,0.0,52.0,10052.0,Australia Remastered: Australia's Oceans,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[['Host Aaron Pedersen']],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,explore incredible diversity ocean wildlife en...
1,0,0.0,60.0,10060.0,Australian Of The Year Profiles 2022,Comedy,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[[]],2022-01-05 17:00:00,G,"[197.0, 210.0, 198.0, 209.0, 181.0, 196.0, 210...","[ABC TV, DOCUMENTARY]","[diversity, australian-capital-territory, tasm...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,celebrate achievement 2022 australian year you...
2,0,0.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
3,0,0.0,163.0,10163.0,Employable Me Australia,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[[]],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[ABC TV, DOCUMENTARY]","[real-life, endearing, informative, disability...",https://cdn.iview.abc.net.au/thumbs/i/dc/DC182...,rose dor nominated series follows six individu...
4,0,0.0,331.0,10331.0,Miriam Margolyes Almost Australian,Documentary,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,[['Host Miriam Margolyes']],2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]","[ABC TV, DOCUMENTARY]","[real-life, irreverent, compelling, ns:be-ente...",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,revealing documentary series following celebra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,32,2.0,266.0,10266.0,Keeping Australia Safe,Documentary,A revealing look at what it takes to keep Aust...,"[In Darwin, a judge decides the fate of a 20-y...",[[]],2022-03-16 23:20:05,M,"[3496.0, 3421.0, 3509.0]","[ABC TV, DOCUMENTARY]","[compelling, aussie, factual, docuseries, orga...",https://cdn.iview.abc.net.au/thumbs/i/do/DO163...,revealing look take keep australian safe sixpa...
419,32,2.0,408.0,10408.0,Prepping Australia,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[[]],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[ABC TV Plus, DOCUMENTARY]","[real-life, factual, docuseries, quirky, unbel...",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,meet real australian preppers bunkering uncert...
420,32,2.0,485.0,10485.0,The Art Of Australia,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[['Host Edmund Capon']],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[informative, a:older-sink-dink, ns:be-inspire...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC123...,art australia explores art artist helped shape...
421,32,1.0,58.0,10058.0,Australia's Ocean Odyssey: A Journey Down The ...,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, s...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,landmark documentary series take 

In [187]:
recommended_div= pd.merge(top3, diversity_library, on=['cluster'], how='left')
recommended_shows_div= pd.merge(top3_show, diversity_library_shows, on=['cluster'], how='left')
recommended_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,7.0,What Was It Like?,240.0,Documentary,What Was It Like?,NaN,What Was It Like?,Eight filmmakers with intellectual disability ...,Eight filmmakers with intellectual disability ...,"[ABC TV, ABC TV Plus, DOCUMENTARY, REGIONAL AU...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[['Director Genevieve Clay-Smith']],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-20 17:01:00,PG,893.0,NaN,,eight filmmaker intellectual disability interv...
2,0,7.0,Re-Frame,248.0,Documentary,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DRAMA, COME...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[['Host Bridie McKim']],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,4 million australian live disability story tel...
3,0,7.0,Re-Frame,277.0,Documentary,Re-Frame 2021,NaN,Re-Frame 2021,"Showcasing the diversity of disability, Re-Fra...","Showcasing the diversity of disability, Re-Fra...","[ABC TV, ABC TV Plus, DOCUMENTARY, REGIONAL AU...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[[]],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-03 07:00:00,M,3493.0,NaN,,showcasing diversity disability reframe brings...
4,0,7.0,Intimate Encounters: Years On,311.0,Comedy,Intimate Encounters: 20 Years On,NaN,Intimate Encounters: 20 Years On,"In 1988, an exhibition caused shockwaves with ...","In 1988, an exhibition caused shockwaves with ...","[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rk/RK209...,[['Director Dieter Knierim']],"[abc1, docs, short, arts, abcarts, sex, disabi...",2020-11-28 14:30:00,M,774.0,NaN,,1988 exhibition caused shockwaves photo people...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,31,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,32,0.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
123,32,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
#drop intersections of recommended and activity by user_id and content_id
recommended = recommended[~recommended[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,14.0,A Few Best Men,23.0,Movies,A Few Best Men,NaN,A Few Best Men,When English lad David announces he is getting...,When English lad David announces he is getting...,"[ABC TV, ABC TV Plus, COMEDY, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW285...,[['Director Stephen Elliott']],"[abc1, abc2, comedy, love, relationships, roma...",2021-06-01 07:00:00,MA,5550.0,NaN,,english lad david announces getting married mi...
4,0,14.0,Cracking COVID,242.0,Documentary,Cracking COVID,NaN,Cracking COVID,Tracking the real-time story of Australia's sc...,Tracking the real-time story of Australia's sc...,"[ABC TV, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO200...,[['Director Sonya Pemberton']],"[abc1, aussie, factual, docs, science, health,...",2021-07-13 20:29:59,PG,4496.0,NaN,,tracking realtime story australia scientific r...
5,0,14.0,Rulla's Connection to Country,274.0,Documentary,Rulla's Connection to Country,NaN,Rulla's Connection to Country,Indigenous Tasmanian man Rulla Kelly-Mansell t...,Indigenous Tasmanian man Rulla Kelly-Mansell t...,"[ABC TV, REGIONAL AUSTRALIA, DOCUMENTARY, INDI...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[['Cast Rulla Kelly-Mansell']],"[abc1, regional, docs, factual, indigenous, ab...",2021-10-13 10:00:00,M,1690.0,NaN,,indigenous tasmanian man rulla kellymansell te...
8,0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,31.0,I'm Wanita,219.0,Documentary,I'm Wanita,NaN,I'm Wanita,"The story of Australia's Queen of Honky Tonk, ...","The story of Australia's Queen of Honky Tonk, ...","[ABC TV Plus, ABC TV, DOCUMENTARY, ARTS & CULT...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC182...,[[]],"[abc2, abc1, docs, factual, abcarts, arts, cul...",2022-04-01 20:32:00,M,3525.0,NaN,,story australia queen honky tonk wanita bahtiy...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,32,8.0,Coppelia,169.0,Family,Coppelia,NaN,Coppelia,"A sparkling tale of magic and mischief, Coppel...","A sparkling tale of magic and mischief, Coppel...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[[]],"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,6511.0,NaN,,sparkling tale magic mischief coppelia everyth...
288,32,8.0,The Merry Widow,297.0,Comedy,The Merry Widow,NaN,The Merry Widow,"The Merry Widow is a lively tale of love, mone...","The Merry Widow is a lively tale of love, mone...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,5877.0,NaN,,merry widow lively tale love money class play ...
290,32,8.0,La Sylphide,312.0,Comedy,La Sylphide,NaN,La Sylphide,"The Australian Ballet presents La Sylphide, a ...","The Australian Ballet presents La Sylphide, a ...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abc2, arts, abcarts, ballet, performance, dance]",2021-06-18 07:00:00,G,3896.0,NaN,,australian ballet present la sylphide ballet t...
292,32,8.0,Warumuk - in the dark night,323.0,Comedy,Warumuk - in the dark night,NaN,Warumuk - in the dark night,A collaboration between Bangarra Dance Theatre...,A collaboration between Bangarra Dance Theatre...,"[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[[]],"[abcarts, arts, ballet, dance, bangarra, first...",2021-06-18 07:00:00,G,2450.0,NaN,,collaboration bangarra dance theatre australia...


In [160]:
recommended_shows = recommended_shows[~recommended_shows[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
1,0,0.0,60.0,10060.0,Australian Of The Year Profiles 2022,Comedy,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[[]],2022-01-05 17:00:00,G,"[197.0, 210.0, 198.0, 209.0, 181.0, 196.0, 210...","[ABC TV, DOCUMENTARY]","[diversity, australian-capital-territory, tasm...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,celebrate achievement 2022 australian year you...
2,0,0.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
4,0,0.0,331.0,10331.0,Miriam Margolyes Almost Australian,Documentary,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,[['Host Miriam Margolyes']],2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]","[ABC TV, DOCUMENTARY]","[real-life, irreverent, compelling, ns:be-ente...",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,revealing documentary series following celebra...
5,0,0.0,495.0,10495.0,The Crown and Us: The Story of The Royals in A...,Documentary,The Crown is part of who we were and who we ar...,[Explore the relationship between Australia an...,[['Director Bernice Toni']],2021-06-18 07:00:00,PG,"[3463.0, 3434.0]","[ABC TV, ABC TV Plus, DOCUMENTARY]","[real-life, informative, nostalgia, factual, c...",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,crown part despite secret scandal question rel...
6,0,2.0,51.0,10051.0,Australia Debates,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[['Host Nikki Britton']],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[ABC TV Plus, PANEL & DISCUSSION]","[ns:be-informed, public-affairs, current-affai...",https://cdn.iview.abc.net.au/thumbs/i/fr/FR201...,funniest mind australia debate finding three i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,32,2.0,210.0,10210.0,Griff's Great Australian Rail Trip,Documentary,British comedian and travel enthusiast Griff R...,[Journeying from Brisbane to Cairns aboard the...,[['Host Griff Rhys Jones']],2021-06-30 07:00:00,G,"[2829.0, 2785.0, 2831.0, 2768.0, 2821.0, 2832.0]","[ABC TV, LIFESTYLE, DOCUMENTARY]","[factual, travel, lifestyle, abc1, uk, austral...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW184...,british comedian travel enthusiast griff rhys ...
418,32,2.0,266.0,10266.0,Keeping Australia Safe,Documentary,A revealing look at what it takes to keep Aust...,"[In Darwin, a judge decides the fate of a 20-y...",[[]],2022-03-16 23:20:05,M,"[3496.0, 3421.0, 3509.0]","[ABC TV, DOCUMENTARY]","[compelling, aussie, factual, docuseries, orga...",https://cdn.iview.abc.net.au/thumbs/i/do/DO163...,revealing look take keep australian safe sixpa...
419,32,2.0,408.0,10408.0,Prepping Australia,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[[]],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[ABC TV Plus, DOCUMENTARY]","[real-life, factual, docuseries, quirky, unbel...",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,meet real australian preppers bunkering uncert...
421,32,1.0,58.0,10058.0,Australia's Ocean Odyssey: A Journey Down The ...,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative, natural-world, animals, s...",https://cdn.iview.abc.net.au/thumbs/i/do/DO180...,land

In [188]:
recommended_div = recommended_div[~recommended_div[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,7.0,What Was It Like?,240.0,Documentary,What Was It Like?,NaN,What Was It Like?,Eight filmmakers with intellectual disability ...,Eight filmmakers with intellectual disability ...,"[ABC TV, ABC TV Plus, DOCUMENTARY, REGIONAL AU...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[['Director Genevieve Clay-Smith']],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-20 17:01:00,PG,893.0,NaN,,eight filmmaker intellectual disability interv...
2,0,7.0,Re-Frame,248.0,Documentary,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DRAMA, COME...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[['Host Bridie McKim']],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,4 million australian live disability story tel...
3,0,7.0,Re-Frame,277.0,Documentary,Re-Frame 2021,NaN,Re-Frame 2021,"Showcasing the diversity of disability, Re-Fra...","Showcasing the diversity of disability, Re-Fra...","[ABC TV, ABC TV Plus, DOCUMENTARY, REGIONAL AU...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[[]],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-03 07:00:00,M,3493.0,NaN,,showcasing diversity disability reframe brings...
4,0,7.0,Intimate Encounters: Years On,311.0,Comedy,Intimate Encounters: 20 Years On,NaN,Intimate Encounters: 20 Years On,"In 1988, an exhibition caused shockwaves with ...","In 1988, an exhibition caused shockwaves with ...","[ABC TV, ABC ARTS, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rk/RK209...,[['Director Dieter Knierim']],"[abc1, docs, short, arts, abcarts, sex, disabi...",2020-11-28 14:30:00,M,774.0,NaN,,1988 exhibition caused shockwaves photo people...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,31,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,32,0.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
123,32,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
recommended_shows_div = recommended_shows_div[~recommended_shows_div[['user_id', 'id']].apply(frozenset, axis=1).isin(activities[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows_div

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
1,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
4,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,31,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,31,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,32,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
97,32,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
#choose 2 rows per each user_id and each cluster
recommended_films = recommended.groupby(['user_id', 'cluster']).head(2)
recommended_films

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,14.0,A Few Best Men,23.0,Movies,A Few Best Men,NaN,A Few Best Men,When English lad David announces he is getting...,When English lad David announces he is getting...,"[ABC TV, ABC TV Plus, COMEDY, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW285...,[['Director Stephen Elliott']],"[abc1, abc2, comedy, love, relationships, roma...",2021-06-01 07:00:00,MA,5550.0,NaN,,english lad david announces getting married mi...
4,0,14.0,Cracking COVID,242.0,Documentary,Cracking COVID,NaN,Cracking COVID,Tracking the real-time story of Australia's sc...,Tracking the real-time story of Australia's sc...,"[ABC TV, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO200...,[['Director Sonya Pemberton']],"[abc1, aussie, factual, docs, science, health,...",2021-07-13 20:29:59,PG,4496.0,NaN,,tracking realtime story australia scientific r...
8,0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,31.0,I'm Wanita,219.0,Documentary,I'm Wanita,NaN,I'm Wanita,"The story of Australia's Queen of Honky Tonk, ...","The story of Australia's Queen of Honky Tonk, ...","[ABC TV Plus, ABC TV, DOCUMENTARY, ARTS & CULT...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC182...,[[]],"[abc2, abc1, docs, factual, abcarts, arts, cul...",2022-04-01 20:32:00,M,3525.0,NaN,,story australia queen honky tonk wanita bahtiy...
10,1,6.0,Bronwyn Oliver: The Shadows Within,243.0,Documentary,Bronwyn Oliver: The Shadows Within,NaN,Bronwyn Oliver: The Shadows Within,This intimate documentary follows the journey ...,This intimate documentary follows the journey ...,"[ABC TV Plus, ABC ARTS, ARTS & CULTURE, DOCUME...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC201...,[[]],"[abc2, arts, abcarts, docs, factual, new-south...",2021-10-13 21:03:54,PG,3567.0,NaN,,intimate documentary follows journey bronwyn o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,32,5.0,Whiteley,128.0,Movies,Whiteley,NaN,Whiteley,A visual journey into the private life and cre...,A visual journey into the private life and cre...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW138...,[['Director James Bogle']],"[abc1, abc2, abcarts, arts, docs, history, bio...",2021-06-01 07:00:00,M,5395.0,NaN,,visual journey private life creative legacy au...
275,32,5.0,Can We Save The Reef?,152.0,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[[]],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,epic story australian international scientist ...
284,32,8.0,Sleeping Beauty,162.0,Family,Sleeping Beauty,NaN,Sleeping Beauty,The Australian Ballet presents Sleeping Beauty...,The Australian Ballet presents Sleeping Beauty...,"[ABC TV Plus, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abc2, arts, entertainment, concert, ballet, c...",2021-06-18 07:00:00,G,7656.0,NaN,,australian ballet present sleeping beauty prod...
286,32,8.0,Coppelia,169.0,Family,Coppelia,NaN,Coppelia,"A sparkling tale of magic and mischief, Coppel...","A sparkling tale of magic and mischief, Coppel...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[[]],"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,6511.0,NaN,,sparkling tale magic mischief coppelia everyth...


In [164]:
recommended_shows = recommended_shows.groupby(['user_id', 'cluster']).head(2)
recommended_shows

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
1,0,0.0,60.0,10060.0,Australian Of The Year Profiles 2022,Comedy,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[[]],2022-01-05 17:00:00,G,"[197.0, 210.0, 198.0, 209.0, 181.0, 196.0, 210...","[ABC TV, DOCUMENTARY]","[diversity, australian-capital-territory, tasm...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,celebrate achievement 2022 australian year you...
2,0,0.0,61.0,10061.0,Australian Story,News,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,[[]],2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....","[ABC TV, ABC NEWS, DOCUMENTARY]","[natural-world, politics, bob-hawke, sport, or...",https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,putting real back reality tv awardwinning seri...
6,0,2.0,51.0,10051.0,Australia Debates,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[['Host Nikki Britton']],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[ABC TV Plus, PANEL & DISCUSSION]","[ns:be-informed, public-affairs, current-affai...",https://cdn.iview.abc.net.au/thumbs/i/fr/FR201...,funniest mind australia debate finding three i...
7,0,2.0,53.0,10053.0,Australia Remastered: Forces of Nature,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[['Host Aaron Pedersen']],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[informative, natural-world, animals, docuseri...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,hosted narrated aaron pedersen series explores...
15,0,5.0,57.0,10057.0,Australia Remastered: Wild Wars,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[['Host Aaron Pedersen']],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[ABC TV, DOCUMENTARY, FAMILY]","[ns:be-informed, natural-world, wildlife, a:ol...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,life australia battle shaped challenge epic en...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,32,0.0,60.0,10060.0,Australian Of The Year Profiles 2022,Comedy,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",[[]],2022-01-05 17:00:00,G,"[197.0, 210.0, 198.0, 209.0, 181.0, 196.0, 210...","[ABC TV, DOCUMENTARY]","[diversity, australian-capital-territory, tasm...",https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,celebrate achievement 2022 australian year you...
412,32,2.0,51.0,10051.0,Australia Debates,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[['Host Nikki Britton']],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[ABC TV Plus, PANEL & DISCUSSION]","[ns:be-informed, public-affairs, current-affai...",https://cdn.iview.abc.net.au/thumbs/i/fr/FR201...,funniest mind australia debate finding three i...
413,32,2.0,53.0,10053.0,Australia Remastered: Forces of Nature,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[['Host Aaron Pedersen']],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[informative, natural-world, animals, docuseri...",https://cdn.iview.abc.net.au/thumbs/i/do/DO184...,hosted narrated aaron pedersen series explores...
421,32,1.0,58.0,10058.0,Australia's Ocean Odyssey: A Journey Down The ...,Family,A landmark documentary series that takes a spe...,[Journey down the East Australian Current all ...,[['Narrator Marta Dusseldorp']],2020-06-09 21:33:25,G,"[3549.0, 360.0, 3515.0, 3446.0]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, informative

In [190]:
recommended_films_div = recommended_div.groupby(['user_id', 'cluster']).head(2)
recommended_films_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features
0,0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,7.0,What Was It Like?,240.0,Documentary,What Was It Like?,NaN,What Was It Like?,Eight filmmakers with intellectual disability ...,Eight filmmakers with intellectual disability ...,"[ABC TV, ABC TV Plus, DOCUMENTARY, REGIONAL AU...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[['Director Genevieve Clay-Smith']],"[abc1, abc2, docs, diversity, cald, disability...",2021-12-20 17:01:00,PG,893.0,NaN,,eight filmmaker intellectual disability interv...
2,0,7.0,Re-Frame,248.0,Documentary,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,"[ABC TV, ABC ARTS, ARTS & CULTURE, DRAMA, COME...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[['Host Bridie McKim']],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,4 million australian live disability story tel...
5,0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,31,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,31,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,32,0.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,"[ABC TV, ABC ARTS, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...
123,32,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
recommended_shows_div = recommended_shows_div.groupby(['user_id', 'cluster']).head(2)
recommended_shows_div

,user_id,cluster,level_0,id,title,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2,image,features
0,0,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
1,0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
4,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,31,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,31,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,32,0.0,572.0,10572.0,Traces,Drama,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]","[ABC TV, DRAMA]","[crime-drama, gritty, mystery, suspense, thril...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,attending online forensic course young lab ass...
97,32,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
#drop na in id column and mix recommended_films and recommended_shows
# recommended_films.dropna(subset=['id'], inplace=True)
# recommended_shows.dropna(subset=['id'], inplace=True)
#join recommended_films and recommended_shows
recommended_mix_austalia = pd.concat([recommended_films, recommended_shows])
#choose the random 6 rows per each user_id
recommended_mix_austalia = recommended_mix_austalia.groupby('user_id').apply(lambda x: x.sample(2, replace=False))
recommended_mix_austalia.reset_index(drop=True, inplace=True)


In [208]:
recommended_mix_austalia.dropna(subset=['id'], inplace=True)
recommended_mix_austalia

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,...,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,level_0
0,0,5.0,NaN,10059.0,Education,Australia's Prime Ministers,NaN,NaN,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",...,https://cdn.iview.abc.net.au/thumbs/i/zx/ZX642...,[[]],"[primary-humanities, education, abc3, gough-wh...",2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...",NaN,NaN,series profile australia leader featuring care...,59.0
1,0,0.0,NaN,10331.0,Documentary,Miriam Margolyes Almost Australian,NaN,NaN,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,...,https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Host Miriam Margolyes']],"[real-life, irreverent, compelling, ns:be-ente...",2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]",NaN,NaN,revealing documentary series following celebra...,331.0
2,1,0.0,NaN,10163.0,Documentary,Employable Me Australia,NaN,NaN,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",...,https://cdn.iview.abc.net.au/thumbs/i/dc/DC182...,[[]],"[real-life, endearing, informative, disability...",2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...",NaN,NaN,rose dor nominated series follows six individu...,163.0
3,1,0.0,NaN,10331.0,Documentary,Miriam Margolyes Almost Australian,NaN,NaN,A revealing documentary series; following cele...,[Miriam is leaving the home she shares with he...,...,https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[['Host Miriam Margolyes']],"[real-life, irreverent, compelling, ns:be-ente...",2021-06-18 07:00:00,M,"[3539.0, 3442.0, 3431.0]",NaN,NaN,revealing documentary series following celebra...,331.0
4,2,0.0,NaN,10061.0,News,Australian Story,NaN,NaN,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,...,https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,[[]],"[natural-world, politics, bob-hawke, sport, or...",2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....",NaN,NaN,putting real back reality tv awardwinning seri...,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,30,0.0,NaN,10061.0,News,Australian Story,NaN,NaN,"Putting the 'real' back into reality TV, the a...",[A daring sea rescue led by the Australian Nav...,...,https://cdn.iview.abc.net.au/thumbs/i/nc/NC210...,[[]],"[natural-world, politics, bob-hawke, sport, or...",2021-08-23 20:31:30,None,"[1856.0, 1920.0, 1837.0, 1782.0, 2070.0, 1693....",NaN,NaN,putting real back reality tv awardwinning seri...,61.0
62,31,0.0,NaN,10060.0,Comedy,Australian Of The Year Profiles 2022,NaN,NaN,Celebrate the achievements of our 2022 Austral...,"[A materials scientist, engineer and inventor ...",...,https://cdn.iview.abc.net.au/thumbs/i/rk/RK210...,[[]],"[diversity, australian-capital-territory, tasm...",2022-01-05 17:00:00,G,"[197.0, 210.0, 198.0, 209.0, 181.0, 196.0, 210...",NaN,NaN,celebrate achievement 2022 australian year you...,60.0
63,31,2.0,NaN,10051.0,Panel Discussion,Australia Debates,NaN,NaN,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",...,https://cdn.iview.abc.net.au/thumbs/i/fr/FR201...,[['Host Nikki Britton']],"[ns:be-informed, public-affairs, current-affai...",2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]",NaN,NaN,funniest mind australia debate finding three i...,51.0
64,32,2.0,NaN,10266.0,Documentary,Keeping Australia Safe,NaN,NaN,A revealing look at what it takes to keep Aust...,"[In Darwin, a judge decides the fate of a 20-y...",...,https://cdn.iview.abc.net.au/thumbs/i/do/DO163...,[[]],"[compelling, aussie, factual, docuseries, orga...",2022-03-16 23:20:05,M,"[3496.0, 3421.0, 3509.0]",NaN,NaN,revealing look take keep australian safe sixpa...,266.0


In [198]:
# recommended_films_div.dropna(subset=['id'], inplace=True)
# recommended_shows_div.dropna(subset=['id'], inplace=True)
recommended_mix_div = pd.concat([recommended_films_div, recommended_shows_div])
#choose the random 6 rows per each user_id
recommended_mix_div = recommended_mix_div.groupby('user_id').apply(lambda x: x.sample(2, replace=False))
recommended_mix_div.reset_index(drop=True, inplace=True)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [195]:
recommended_mix_div.dropna(subset=['id'], inplace=True)

In [206]:
recommended_mix_div

,user_id,cluster,episode_title,id,category,title,series,episode_name,description,description2,...,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,features,level_0
2,1,7.0,Re-Frame,248.0,Documentary,Re-Frame 2020,NaN,Re-Frame 2020,More than 4 million Australians live with a di...,More than 4 million Australians live with a di...,...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[['Host Bridie McKim']],"[abc1, disability, arts, abcarts, drama, comed...",2020-11-28 14:30:00,M,3480.0,NaN,,4 million australian live disability story tel...,NaN
4,2,0.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,...,https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...,NaN
5,2,4.0,The Broken Shore,92.0,Movies,The Broken Shore,NaN,The Broken Shore,An evocative crime thriller set against a back...,An evocative crime thriller set against a back...,...,https://cdn.iview.abc.net.au/thumbs/i/dr/DR121...,[['Director Rowan Woods']],"[abc1, abc2, drama, crime, film, murder, aussi...",2021-06-11 07:05:00,M,6212.0,NaN,,evocative crime thriller set background police...,NaN
6,3,4.0,Capturing Cricket: Steve Waugh In India,232.0,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...",...,https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[['Host Steve Waugh']],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,steve waugh mastermind behind australia golden...,NaN
8,4,0.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
13,6,0.0,Australia Day Live,313.0,Comedy,Australia Day Live 2022,NaN,Australia Day Live 2022,A concert and fireworks spectacular from Sydne...,A concert and fireworks spectacular from Sydne...,...,https://cdn.iview.abc.net.au/thumbs/i/rv/RV210...,"[['Host Jeremy Fernandez', 'Casey Donovan', 'J...","[abc1, aussie, australia, event, concert, musi...",2022-01-26 21:31:00,PG,7871.0,NaN,,concert firework spectacular sydney featuring ...,NaN
15,7,0.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
16,8,0.0,NaN,10572.0,Drama,Traces,NaN,NaN,"While attending an online forensic course, a y...",[While Kathy oversees the excavation of Marie'...,...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW296...,"[['Cast Molly Windsor', 'Jennifer Spence', 'La...","[crime-drama, gritty, mystery, suspense, thril...",2021-08-15 21:27:00,M,"[2618.0, 2592.0, 2712.0, 2642.0, 2581.0, 2593.0]",NaN,NaN,attending online forensic course young lab ass...,572.0
21,10,1.0,Ochres,326.0,Comedy,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...",...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[[]],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,told five part ochre inspired special role och...,NaN
23,11